In [12]:
import gizmo_analysis as gizmo
import h5py
import halo_analysis as halo
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.interpolate as interp
import scipy.signal as signal
import utilities as ut
from gc_utils import iteration_name, snapshot_name  # type: ignore

In [2]:
sim = "m12i"
sim_dir = "/Users/z5114326/Documents/simulations/"
fire_dir = sim_dir + sim + "/" + sim + "_res7100"

proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

snap = fire_dir + "/snapshot_times.txt"
snap_data = pd.read_table(snap, comment="#", header=None, sep=r"\s+")
snap_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]


Get COM, and SIGMA of GC system in relation to its host galaxy (pre accretion) and then the MW (post accretion)

In [17]:
halt = halo.io.IO.read_tree(simulation_directory=fire_dir)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt


# in halo_analysis.halo_io.IO():
* read 17859038 halos from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/halo/rockstar_dm/catalog_hdf5/tree.hdf5



In [114]:
def get_host_at_snap(grp_val, snap_final, acc_snap, halt):
    if grp_val != 0:
        idx = np.where(halt["tid"] == grp_val)[0][0]
        snap = halt["snapshot"][idx]

        if snap < acc_snap:
            while snap > snap_final:
                idx = halt["progenitor.main.index"][idx]
                snap = halt["snapshot"][idx]

        else:
            idx = 0
            snap = halt["snapshot"][idx]

            while snap > snap_final:
                idx = halt["progenitor.main.index"][idx]
                snap = halt["snapshot"][idx]

    else:
        idx = 0
        snap = halt["snapshot"][idx]

        while snap > snap_final:
            idx = halt["progenitor.main.index"][idx]
            snap = halt["snapshot"][idx]

    host_tid = halt["tid"][idx]

    # print(snap, host_tid)

    return host_tid, idx

In [345]:
it = 0
snapshot = 77

it_id = iteration_name(it)
snap_id = snapshot_name(snapshot)

gc_ids = proc_data[it_id]["snapshots"][snap_id]["gc_id"][()]
ptypes = proc_data[it_id]["snapshots"][snap_id]["ptype"][()]

grp_lst = proc_data[it_id]["snapshots"][snap_id]["group_id"][()]
grp_vals = np.unique(np.abs(grp_lst))

test_idx = -1

snap_idx = np.where(np.abs(grp_lst) == grp_vals[test_idx])[0][0]
acc_snap = proc_data[it_id]["snapshots"][snap_id]["acc_snap"][snap_idx]

host_tid, host_index = get_host_at_snap(grp_vals[test_idx], snapshot, acc_snap, halt)
host_tid, host_index, acc_snap, snapshot

(594934, 2106994, 554, 77)

In [194]:
halt = halo.io.IO.read_tree(simulation_directory=fire_dir, assign_hosts_rotation=True)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

* read 17859038 halos from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/halo/rockstar_dm/catalog_hdf5/tree.hdf5


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  read 1 host (position, velocity, principal axes) from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/track/host_coordinates.hdf5
  host1 position = (41792.15, 44131.23, 46267.68) [kpc comoving]
  host1 velocity = (-52.2, 73.6, 94.7) [km/s]
  host1 axis ratios = (0.16, 0.16, 0.99)


In [150]:
halt.prop("host.distance", 0)

array([0., 0., 0.], dtype=float32)

In [153]:
halt.prop("host.distance", host_index)

array([-58.441227, -84.7847  , -55.638245], dtype=float32)

In [158]:
proc_data[it_id]["snapshots"][snap_id]["z"][snap_idx]

-126.11447698282915

In [309]:
part = gizmo.io.Read.read_snapshots(
    ["star", "dark"],
    "index",
    77,
    simulation_directory=fire_dir,
    host_number=210,
    assign_hosts="mass",
    assign_hosts_rotation=True,
)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 77, redshift = 4.478


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/output/snapdir_077/snapshot_077.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 70388512 particles
    star      (id = 4): 125821 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['star', 'dark']

* reading particles from:
    snapshot_077.0.hdf5
    snapshot_077.1.hdf5
    snapshot_077.2.hdf5
    snapshot_077.3.hdf5

* reading cosmological parameters from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in utilities.particle.get_center_positions():
* assigni

In [205]:
halt_mask = halt["snapshot"] == 77
halt_masses = np.flip(np.sort(halt["mass"][halt_mask]))
halt_masses

array([2.6826209e+10, 2.2678063e+10, 2.0649572e+10, ..., 5.1364670e+06,
       5.1012820e+06, 5.1012820e+06], dtype=float32)

In [206]:
host_mass = halt["mass"][host_index]
np.where(halt_masses == host_mass)[0][0]

209

In [212]:
host_name = ut.catalog.get_host_name(209, "")
host_name

'host210'

In [208]:
ptype = ptypes[snap_idx].decode("utf-8")
gc_id = gc_ids[snap_idx]

gc_id, ptype

(103424386, 'dark')

In [344]:
part_idx = np.where(part[ptype]["id"] == gc_id)[0][0]
part_idx

25890207

In [235]:
part.host.keys()

dict_keys(['position', 'velocity', 'acceleration', 'rotation', 'axis.ratios'])

In [239]:
part[ptype]["position"][part_idx]

array([43081.26877916, 41478.92367249, 44031.41436336])

In [240]:
part[ptype].prop("host.distance.principal", part_idx)

array([153.31440515, 190.27675639, -55.53828799])

In [288]:
galaxy_pos = part.host["position"][0]
galaxy_vel = part.host["velocity"][0]
galaxy_rotation = part.host["rotation"][0]


test_0 = ut.particle.get_distances_wrt_center(
    part, species=["dark"], center_position=galaxy_pos, rotation=galaxy_rotation
)

In [289]:
test_0[part_idx]

array([153.31440515, 190.27675639, -55.53828799])

In [290]:
galaxy_pos = part.host["position"][209]
galaxy_vel = part.host["velocity"][209]
galaxy_rotation = part.host["rotation"][209]


test_209 = ut.particle.get_distances_wrt_center(
    part, species=["dark"], center_position=galaxy_pos, rotation=galaxy_rotation
)

In [291]:
test_209[part_idx]

array([168.89218873,  52.24633681,  64.73138206])

In [303]:
part.host["position"][0] - part.host["position"][209]

array([-165.75426841,  340.505654  ,  563.94955095])

In [306]:
# halt["host.distance"][2105151]
halt["position"][host_index]

array([43082.61 , 41478.746, 44032.11 ], dtype=float32)

In [307]:
part.host["position"][209]

array([43435.13358152, 42435.24690565, 43876.62582255])

In [313]:
halt["position"][host_index]

array([43082.61 , 41478.746, 44032.11 ], dtype=float32)

In [373]:
part = gizmo.io.Read.read_snapshots(
    ["dark", "star", "gas", "dark2"],
    "index",
    77,
    simulation_directory=fire_dir,
    assign_hosts_rotation=True,
)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 77, redshift = 4.478


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/output/snapdir_077/snapshot_077.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 70388512 particles
    star      (id = 4): 125821 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['dark', 'star', 'gas', 'dark2']

* reading particles from:
    snapshot_077.0.hdf5
    snapshot_077.1.hdf5
    snapshot_077.2.hdf5
    snapshot_077.3.hdf5

* reading cosmological parameters from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.ParticleC

In [374]:
dist = ut.particle.get_distances_wrt_center(
    part,
    species=["dark", "star", "gas"],
    center_position=part.host["position"],
    rotation=True,
)

In [375]:
dist["dark"][part_idx]

array([ -25.59074595,  -14.93868734, -126.11447698])

In [376]:
(
    proc_data[it_id]["snapshots"][snap_id]["x"][snap_idx],
    proc_data[it_id]["snapshots"][snap_id]["y"][snap_idx],
    proc_data[it_id]["snapshots"][snap_id]["z"][snap_idx],
)

(-25.59074594971148, -14.938687337122902, -126.11447698282915)

In [394]:
dist = ut.particle.get_distances_wrt_center(
    part,
    species=["dark", "star", "gas"],
    center_position=halt["position"][host_index],
    rotation=True,
    total_distance=True,
)

dist_vect = ut.particle.get_distances_wrt_center(
    part,
    species=["dark", "star", "gas"],
    center_position=halt["position"][host_index],
    rotation=True,
)

vel_vect = ut.particle.get_velocities_wrt_center(
    part,
    species=["dark", "star", "gas"],
    center_position=halt["position"][host_index],
    center_velocity=halt["velocity"][host_index],
    rotation=True,
)

In [444]:
sp = "star"
rmax_ctr = 10

ctr_indices = np.where(dist[sp] < rmax_ctr)[0]

m = part[sp]["mass"][ctr_indices]
pos = part[sp]["position"][ctr_indices]
vel = part[sp]["velocity"][ctr_indices]
new_ctr = np.multiply(m, pos.T).sum(axis=1) / np.sum(m)
new_vctr = np.multiply(m, vel.T).sum(axis=1) / np.sum(m)

In [462]:
new_rot = ut.particle.get_principal_axes(
    part,
    species_name="star",
    part_indicess=ctr_indices,
    center_positions=new_ctr,
    center_velocities=new_vctr,
)


# in utilities.particle.get_principal_axes():
* assigning rotation tensor for 1 host
  using star particles at distance < 10.0 kpc
  host1: axis ratios: min/maj = 0.576, min/med = 0.797, med/maj = 0.723



In [480]:
dist_vect_new = ut.particle.get_distances_wrt_center(
    part,
    species=["dark", "star"],
    center_position=new_ctr,
    rotation=new_rot["rotation"],
)

vel_vect_new = ut.particle.get_velocities_wrt_center(
    part,
    species=["dark", "star"],
    center_position=new_ctr,
    center_velocity=new_vctr,
    rotation=new_rot["rotation"],
)

vel_vect_cyl_new = ut.particle.get_velocities_wrt_center(
    part,
    species=["dark", "star"],
    center_position=new_ctr,
    center_velocity=new_vctr,
    rotation=new_rot["rotation"],
    coordinate_system="cylindrical",
)

In [478]:
np.linalg.norm(vel_vect_new[ptype][part_idx])

15.930299

In [481]:
np.linalg.norm(vel_vect_cyl_new[ptype][part_idx])

15.9303